In [ ]:
import os
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

import cv2
from ultralytics import YOLO

# -----------------------
# GPU 확인
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"[INFO] {len(gpus)} GPU(s) available: {[gpu.name for gpu in gpus]}")
else:
    print("[INFO] No GPU available. Training will use CPU.")

# -----------------------
# Config
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 15,
    'LR': 3e-4,
    'BATCH_SIZE': 24,
    'SEED': 2025,
    'FOLDS': 5,
    'FREEZE_EPOCHS': 5,
    'UNFREEZE_AT': 50  # EfficientNetB0 layer index to unfreeze from
}

# Seed 고정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(CFG['SEED'])

# -----------------------
# 경로 설정
ORIGINAL_TRAIN_DIR = "D:/데이콘 250519 대회/open/train"
FILTERED_TRAIN_DIR = "D:/데이콘 250519 대회/filtered_train"
TEST_DIR = "D:/데이콘 250519 대회/open/test"
SAMPLE_SUB = "D:/데이콘 250519 대회/open/sample_submission.csv"

# -----------------------
# YOLOv8 차량 외관 필터링 함수

VEHICLE_CLASSES = [2, 5, 7]  # car, bus, truck 클래스 번호

yolo_model = YOLO('yolov8n.pt')

def is_full_vehicle(detections, img):
    h, w = img.shape[:2]
    img_area = h * w

    for det in detections:
        cls = int(det.cls)
        if cls not in VEHICLE_CLASSES:
            continue

        x1, y1, x2, y2 = det.xyxy[0].cpu().numpy()
        box_w, box_h = x2 - x1, y2 - y1
        area = box_w * box_h
        center_x, center_y = (x1 + x2) / 2, (y1 + y2) / 2
        aspect_ratio = box_w / box_h

        # 차량 박스 크기 최소 비율 조건 강화 (0.35 → 0.4 이상)
        if area / img_area < 0.4:
            continue

        # 차량 비율 조건 (트렁크/보닛 열린 이미지 제외용)
        if aspect_ratio < 0.9 or aspect_ratio > 2.0:
            continue

        # 차량 중앙 근처 위치 조건 엄격히 조정
        if not (0.4 * w < center_x < 0.6 * w and 0.4 * h < center_y < 0.6 * h):
            continue

        # 추가 조건: 박스가 이미지 가장자리와 너무 가까우면 제외 (외곽 확대 이미지 제외)
        margin = 0.05
        if (x1 < margin * w) or (y1 < margin * h) or (x2 > (1 - margin) * w) or (y2 > (1 - margin) * h):
            continue

        # 위 조건 모두 통과 시 전체 차량 외관 포함으로 판단
        return True

    return False

# -----------------------
# 차량 외관 필터링 수행

print("차량 외관 이상치 필터링 시작...")
os.makedirs(FILTERED_TRAIN_DIR, exist_ok=True)
class_dirs = sorted(os.listdir(ORIGINAL_TRAIN_DIR))

filtered_count = 0
total_count = 0

for cls in tqdm(class_dirs, desc="클래스별 필터링"):
    input_dir = os.path.join(ORIGINAL_TRAIN_DIR, cls)
    output_dir = os.path.join(FILTERED_TRAIN_DIR, cls)
    os.makedirs(output_dir, exist_ok=True)

    image_paths = glob(os.path.join(input_dir, "*.jpg"))
    for img_path in image_paths:
        total_count += 1
        img = cv2.imread(img_path)
        results = yolo_model(img, verbose=False)[0]

        if is_full_vehicle(results.boxes, img):
            filename = os.path.basename(img_path)
            cv2.imwrite(os.path.join(output_dir, filename), img)
            filtered_count += 1

print(f" 전체 이미지 수: {total_count}")
print(f" 필터링된 차량 외관 이미지 수: {filtered_count}")
print(f"🧼 이상치 제거 완료. 정제 비율: {filtered_count / total_count:.2%}")

# -----------------------
# 학습 데이터 준비

label_list = sorted(os.listdir(FILTERED_TRAIN_DIR))
label2id = {v: i for i, v in enumerate(label_list)}
id2label = {i: v for v, i in label2id.items()}

image_paths = glob(os.path.join(FILTERED_TRAIN_DIR, '*', '*.jpg'))
labels = [label2id[os.path.basename(os.path.dirname(p))] for p in image_paths]

# -----------------------
# 데이터 전처리 및 증강 함수

def load_and_preprocess(img_path):
    img = load_img(img_path, target_size=(CFG['IMG_SIZE'], CFG['IMG_SIZE']))
    img = img_to_array(img)
    img = preprocess_input(img)
    return img

def data_augmentation(image, label):
    # TensorFlow 텐서로 변환 가정
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    image = tf.image.random_hue(image, max_delta=0.05)
    image = tf.image.random_rotation(image, angles=0.1) if hasattr(tf.image, 'random_rotation') else image  # tf.image에 없을 수도 있음

    return image, label

def create_dataset(image_paths, labels=None, is_train=True):
    def gen():
        for i, path in enumerate(image_paths):
            img = load_and_preprocess(path)
            if labels is not None:
                yield img, labels[i]
            else:
                yield img

    if labels is not None:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=(tf.float32, tf.int32),
            output_shapes=((CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3), ())
        )
    else:
        ds = tf.data.Dataset.from_generator(
            gen,
            output_types=tf.float32,
            output_shapes=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3)
        )

    if is_train and labels is not None:
        ds = ds.map(data_augmentation, num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.shuffle(1024)
    ds = ds.batch(CFG['BATCH_SIZE']).prefetch(tf.data.AUTOTUNE)
    return ds

# -----------------------
# 모델 생성 + Fine-tuning 준비

def build_model(num_classes, freeze_base=True):
    base = tf.keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(CFG['IMG_SIZE'], CFG['IMG_SIZE'], 3),
        weights='imagenet',
        pooling='avg'
    )
    if freeze_base:
        base.trainable = False
    else:
        # 일부 레이어 unfreeze (UNFREEZE_AT 이후)
        base.trainable = True
        for layer in base.layers[:CFG['UNFREEZE_AT']]:
            layer.trainable = False

    x = layers.Dense(num_classes, activation='softmax')(base.output)
    model = models.Model(inputs=base.input, outputs=x)
    return model

# -----------------------
# 학습 콜백

lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=2, verbose=1, min_lr=1e-6
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss', patience=4, restore_best_weights=True, verbose=1
)

# -----------------------
# Stratified K-Fold 학습 (freeze -> unfreeze)

skf = StratifiedKFold(n_splits=CFG['FOLDS'], shuffle=True, random_state=CFG['SEED'])

for fold, (train_idx, val_idx) in enumerate(skf.split(image_paths, labels)):
    print(f"\n### Fold {fold+1} 시작 ###")

    train_paths = [image_paths[i] for i in train_idx]
    val_paths = [image_paths[i] for i in val_idx]
    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]

    train_ds = create_dataset(train_paths, train_labels, is_train=True)
    val_ds = create_dataset(val_paths, val_labels, is_train=False)

    # 1) Freeze base model
    model = build_model(num_classes=len(label2id), freeze_base=True)
    model.compile(
        optimizer=optimizers.Adam(CFG['LR']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    print("1) Base 모델 Freeze 상태로 학습 시작")
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=CFG['FREEZE_EPOCHS'],
        callbacks=[lr_scheduler, early_stopping],
        verbose=1
    )

    # 2) Unfreeze 일부 레이어로 Fine-tuning
    model = build_model(num_classes=len(label2id), freeze_base=False)
    model.compile(
        optimizer=optimizers.Adam(CFG['LR'] / 10),  # LR 낮춰서 fine-tuning
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    print("2) 일부 레이어 Unfreeze 후 Fine-tuning 시작")
    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=CFG['EPOCHS'] - CFG['FREEZE_EPOCHS'],
        callbacks=[lr_scheduler, early_stopping],
        verbose=1
    )

# -----------------------
# 테스트 데이터 추론 및 제출 파일 생성

test_paths = sorted(glob(os.path.join(TEST_DIR, '*.jpg')))
test_ds = create_dataset(test_paths, is_train=False)

print("테스트 데이터 추론 시작...")
preds = model.predict(test_ds)
print("추론 완료")

submission = pd.read_csv(SAMPLE_SUB)
for idx, class_name in enumerate(label2id.keys()):
    submission[class_name] = preds[:, idx]

submission.to_csv("submission.csv", index=False)
print("submission.csv 저장 완료")


[INFO] 1 GPU(s) available: ['/physical_device:GPU:0']
차량 외관 이상치 필터링 시작...


클래스별 필터링:  14%|█▍        | 56/396 [04:59<30:24,  5.37s/it]